# extract

> The extraction engine: `extract_categorical` for pulling stats from any categorical raster.

In [1]:
#| default_exp extract

In [1]:
#| hide
from nbdev.showdoc import *

In [18]:
#| export
import ee
import geemap
import pandas as pd
from fastcore.basics import patch
from typing import Optional

from gee_polygons.layers import CategoricalLayer
from gee_polygons.site import Site

## The Core Primitive

The heart of gee-polygons is one powerful primitive:

```python
site.extract_categorical(layer, years=[2018, 2019, 2020])
```

Given:
- A polygon (Site)
- A categorical raster descriptor (CategoricalLayer)
- A list of years

It returns a tidy DataFrame with pixel counts and areas per class, per year.

**This function knows nothing about MapBiomas, deforestation, or Brazil.** It's pure geometry + categorical values + time.

In [19]:
#| export
@patch
def extract_categorical(
    self: Site,
    layer: CategoricalLayer,
    years: list[int],
    max_pixels: int = int(1e9)
) -> pd.DataFrame:
    """Extract per-year pixel counts and areas from a categorical raster.
    
    Args:
        layer: A CategoricalLayer describing the data source
        years: List of years to extract
        max_pixels: Maximum pixels for reduction (default 1e9)
        
    Returns:
        A tidy DataFrame with columns:
        - site_id: Site identifier
        - year: Year of observation
        - class_value: Integer class value
        - count: Pixel count
        - area_ha: Area in hectares
        - class_name: Human-readable name (if available in layer)
    """
    img = ee.Image(layer.asset_id)
    records = []
    
    for year in years:
        band = layer.band_name(year)
        classified = img.select(band)
        
        stats = classified.reduceRegion(
            reducer=ee.Reducer.frequencyHistogram(),
            geometry=self.geometry,
            scale=layer.scale,
            maxPixels=max_pixels
        )
        
        hist = ee.Dictionary(stats.get(band))
        
        # Build a feature to batch the getInfo call
        records.append(
            ee.Feature(None, {
                'year': year,
                'histogram': hist
            })
        )
    
    # Single getInfo call for all years
    fc = ee.FeatureCollection(records).getInfo()
    
    # Convert to tidy rows
    rows = []
    for f in fc['features']:
        year = f['properties']['year']
        hist = f['properties']['histogram']
        
        if hist is None:
            continue
            
        for cls_str, count in hist.items():
            cls = int(cls_str)
            rows.append({
                'site_id': self.site_id,
                'year': year,
                'class_value': cls,
                'count': count,
                'area_ha': count * (layer.scale ** 2) / 10_000,
                'class_name': layer.class_name(cls)
            })
    
    return pd.DataFrame(rows)

## Example Usage

Let's test with a simple example. First, we need to initialize GEE and load a site.

In [20]:
# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project='hs-brazilreforestation')

In [44]:
from gee_polygons.site import load_sites
sites = load_sites('../data/restoration_sites_subset.geojson')
site = sites[8]
print(site)

Site(id=9368, start_year=2012)


In [47]:
# Define a layer (or use a preset from gee_polygons.datasets)
layer = CategoricalLayer(
     asset_id='projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2',
     band_pattern='classification_{}',
     scale=30
)

# Extract stats
df = site.extract_categorical(layer, years=[2012, 2013, 2014, 2015])
df.head(10)

,site_id,year,class_value,count,area_ha,class_name
0,9368,2012,21,147.211765,13.249059,None
1,9368,2012,3,9.196078,0.827647,None
2,9368,2013,21,99.458824,8.951294,None
3,9368,2013,3,56.949020,5.125412,None
4,9368,2014,21,86.427451,7.778471,None
5,9368,2014,3,69.980392,6.298235,None
6,9368,2015,3,156.407843,14.076706,None


The output is always a tidy DataFrame:

| site_id | year | class_value | count | area_ha | class_name |
|---------|------|-------------|-------|---------|------------|
| 3107    | 2020 | 3           | 1520  | 136.8   | None       |
| 3107    | 2020 | 5           | 203   | 18.3    | None       |
| 3107    | 2021 | 3           | 1701  | 153.1   | None       |

If the `CategoricalLayer` has a `class_map`, the `class_name` column will be populated.

We can also try with a pre-set dataset.

In [48]:
from gee_polygons.datasets.mapbiomas import MAPBIOMAS_DEFREG
df = site.extract_categorical(MAPBIOMAS_DEFREG, years=range(2012, 2015))
df.head(10)

,site_id,year,class_value,count,area_ha,class_name
0,9368,2012,1,147.211765,13.249059,Anthropic
1,9368,2012,2,0.462745,0.041647,Primary Vegetation
2,9368,2012,3,8.733333,0.786000,Secondary Vegetation
3,9368,2013,1,99.458824,8.951294,Anthropic
4,9368,2013,2,0.462745,0.041647,Primary Vegetation
5,9368,2013,3,8.733333,0.786000,Secondary Vegetation
6,9368,2013,5,47.752941,4.297765,Secondary Veg Regrowth
7,9368,2014,1,86.427451,7.778471,Anthropic
8,9368,2014,2,0.462745,0.041647,Primary Vegetation
9,9368,2014,3,56.486275,5.083765,Secondary Vegetation


## Visualizing Layers

Before extracting stats, it's useful to visually verify the data. The `show_layer` method displays the categorical raster for specified years with the site polygon overlaid.

In [49]:
#| export
@patch
def show_layer(
    self: Site,
    layer: CategoricalLayer,
    years: list[int],
    zoom: int = 14,
    basemap: str = 'SATELLITE',
    site_color: str = 'blue',
    buffer_m: Optional[float] = None
) -> geemap.Map:
    """Display a categorical layer for multiple years with the site overlaid.
    
    Args:
        layer: A CategoricalLayer to visualize
        years: List of years to add as layers
        zoom: Initial zoom level (default 14)
        basemap: Basemap type (default 'SATELLITE')
        site_color: Color for site boundary (default 'blue')
        buffer_m: Optional buffer around site for clipping display
        
    Returns:
        A geemap.Map with yearly classification layers
    """
    m = geemap.Map()
    m.add_basemap(basemap)
    m.center_object(self.geometry, zoom)
    
    img = ee.Image(layer.asset_id)
    
    # Determine clip geometry
    clip_geom = self.buffer(buffer_m) if buffer_m else self.geometry
    
    # Build visualization params from palette
    if layer.palette:
        # Get all class values and their colors in order
        classes = sorted(layer.palette.keys())
        colors = [layer.palette[c] for c in classes]
        vis_params = {
            'min': min(classes),
            'max': max(classes),
            'palette': colors
        }
    else:
        # Default grayscale
        vis_params = {'min': 0, 'max': 10}
    
    # Add each year as a layer (most recent visible by default)
    for i, year in enumerate(years):
        band = layer.band_name(year)
        classified = img.select(band).clip(clip_geom)
        
        # Only show the last year by default
        shown = (i == len(years) - 1)
        m.add_layer(classified, vis_params, f'{year}', shown=shown)
    
    # Add site boundary on top
    site_style = {'color': site_color, 'fillColor': '#00000000', 'width': 2}
    m.add_layer(self.geometry, site_style, f'Site {self.site_id}')
    
    return m

In [51]:
# Example: Visualize the layer with a buffer around the site
site.show_layer(MAPBIOMAS_DEFREG, years=range(2010, 2018), buffer_m=500)

Map(center=[-22.511306816393418, -42.27634385094392], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()